setup

In [ ]:
import pandas as pd
import re
import json

df = pd.read_csv("raw data/cpd extract janaka.js", sep="\t", header=None)
df

clean up

In [ ]:

length =df.shape[0]
width = df.shape[1]

for row in range(length): # length:
	for column in range(width):

		cell = df.iloc[row, column]
		cell = cell.replace("ṁ", "ṃ")
		cell = cell.replace("\\'", "'")
		df.iloc[row, column] = cell


add columns and rename

In [ ]:
df.insert(2, "definition_plain", "")
df.insert(3, "synonyms", "")
df.rename({0:"word", 1:"definition_html"}, axis='columns', inplace=True)

df.to_json("output/cpd.json", force_ascii=False, orient="records", indent=5)
df

generate goldendict

In [ ]:
from typing import List
from pathlib import Path
from simsapa.app.stardict import export_words_as_stardict_zip, ifo_from_opts, DictEntry
import json

zip_path = Path("./output/cpd.zip")

with open("output/cpd.json", "r") as gd_data:
    data_read = json.load(gd_data)

def item_to_word(x):
    return DictEntry(
        word=x["word"],
        definition_html=x["definition_html"],
        definition_plain=x["definition_plain"],
        synonyms=x["synonyms"],
    )

words = list(map(item_to_word, data_read))

ifo = ifo_from_opts(
    {
        "bookname": "cpd",
        "author": "",
        "description": "critical pāli dictionary",
        "website": "https://cpd.uni-koeln.de",
    }
)

export_words_as_stardict_zip(words, ifo, zip_path)